# Tutotial DataReader

Utilisation des objets DataReader pour la lectures de données sources spécifiques. 
En résumé, que font les DataReader:
- Prise en charge d'un input specifique (une source de donnée comme par expl otoriver ou sensea).
- Prétraitmements éventuels sur la donnée brut (expl: passer sur un pas de temps horaire).
- Construction d'un dataset en sortie

In [ ]:
import os
cwd = os.getcwd()
new_cwd = os.path.join(cwd.rsplit("SeinAcoustic_DataAnalysis")[0], "SeinAcoustic_DataAnalysis") if not cwd.endswith("SeinAcoustic_DataAnalysis") else None
if new_cwd != None:
    os.chdir(new_cwd)

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import glob

In [ ]:
def min_max_normalize(column):
    return (column - column.min()) / (column.max() - column.min())

def normalize_dataframe (df:pd.DataFrame, columns): 
    df_norm = df.copy()
    df_norm[columns] = df_norm[columns].apply(min_max_normalize, axis=0)
    return df_norm

In [ ]:
def get_lines (df:pd.DataFrame, x:str, y:any, color_palette="hls"):
    traces = []
    y_list = [y] if type(y) == str else y
    colors = sns.color_palette(color_palette, n_colors=len(y_list)).as_hex()
    for i in range(len(y_list)):
        y = y_list[i]
        traces.append(go.Scatter(x=df[x], y=df[y], mode='lines', name=y, line=dict(color=colors[i])))
    return traces

In [ ]:
def line_graph (df:pd.DataFrame, x:str, y:any, title:str=None, xaxis_title:str=None, yaxis_title:str=None, color_palette="hls", norm:bool=False):
    """
    tarce a line graph from dataframe
    """
    if norm:
        df = normalize_dataframe(df, columns=y)
    fig = go.Figure(
        data=get_lines(df=df, x=x, y=y, color_palette=color_palette)
    )
    args_layout={}
    if title:
        args_layout["title"] = title
    if xaxis_title:
        args_layout["xaxis_title"] = xaxis_title
    if yaxis_title:
        args_layout["yaxis_title"] = yaxis_title
    if len(args_layout) > 0:
        fig.update_layout(**args_layout)
    return fig

## 1. Descripteurs Acoustiques: 

Lecture des données d'entrée sur les descripteurs acoustiques. 
2 sources de données:
- Sensea : 11 descripteurs => note : manque de métadonnées (seul info = unité = comptage sur les données de sortie du DataReader)
- Otoriver : 21 descripteurs => unités différentes en fonction des descripteurs (métadonnée complette) - a voir pout adapter le traitmeent en fonction des unités.

In [ ]:
from DBBuilder.datareader import Otoriver_DataReader

# Initialisation de du DataReader
datareader_otoriver = Otoriver_DataReader(timefreq="h") # Choisire le pas de temps de sortie
# datareader_otoriver.interpolate_option = True
metadata_otoriver = datareader_otoriver.metadata # obtenir les métadonnées disponnible sur otoriver dans l'outil DataReader
metadata_otoriver

In [ ]:
# Données brut otoriver
filepath = "data/Data_SAM/Bougival/input/acoustique/otoriver/otoriver_20210706-20231208.csv" 
df_otoriver_input = pd.read_csv(filepath) 
print("Données brut pour otoriver")
df_otoriver_input

In [ ]:
fig_otoriver_input = line_graph(df_otoriver_input, x="date", y=metadata_otoriver["colname"].values.tolist(), title="Evolution temporelle de l'activité acoustiques par les descripteurs d'Otoriver (Données d'entrée)")
fig_otoriver_input.show()

In [ ]:
df_otoriver_output = datareader_otoriver.from_csv(filepath=filepath) # Lecture du fichier à l'aide du dataReader
print("Fichier de sortie pour otoriver")
df_otoriver_output

In [ ]:
otoriver_descriptors = [c for c in df_otoriver_output.columns if c != "date"]
fig_otoriver = line_graph(df_otoriver_output, x="date", y=otoriver_descriptors, title="Evolution temporelle de l'activité acoustiques par les descripteurs d'Otoriver")
fig_otoriver.show()

In [ ]:
figs = {}
metadata_otoriver = Otoriver_DataReader.metadata
otoriver_descriptors = [c for c in df_otoriver_output.columns if c != "date"]
for unit in metadata_otoriver["unit"].unique():
    # y = [c for c in otoriver_descriptors if unit in c[c.index("[")+1:c.index("]")].split(",")]
    y = metadata_otoriver.loc[(metadata_otoriver["unit"] == unit), "colname"].values.tolist()
    figs[unit]=line_graph(df_otoriver_output, x="date", y=y, title=f"Evolution de l'activité acoustiques des descripteurs Otoriver (type d'unité {unit})")
    figs[unit].show()

In [ ]:
# df_otoriver_output_norm = normalize_acoustic(df_otoriver_output)
fig_otoriver = line_graph(df_otoriver_output, x="date", y=otoriver_descriptors, norm=True, title="Evolution temporelle de l'activité acoustiques par les descripteurs d'Otoriver (avec normalisation)")
fig_otoriver.show()

## 2. Descripteurs Acoustiques de Sensea : Sensea DataReader

Permettre la lecture et le prétraitement des données sensea

In [ ]:
from DBBuilder.datareader import Sensea_DataReader

# Initialisation de du DataReader
datareader_sensea = Sensea_DataReader(timefreq="15min") # Choisire le pas de temps de sortie
datareader_sensea.metadata # obtenir les métadonnées disponnible sur sensea dans l'outil DataReader

In [ ]:
filepath_sensea = "data/Data_SAM/Bougival/input/acoustique/sensea/sensea_2021-06-16_2023-11-21.csv"
df_sensea_input = pd.read_csv(filepath_sensea)
print("Données brut pour sensea")
df_sensea_input

In [ ]:
df_sensea_output = datareader_sensea.from_dataframe(df=df_sensea_input) # Lecture du fichier à l'aide du dataReader
print("Fichier de sortie pour sensea")
df_sensea_output

In [ ]:
descriptors_sensea = [c for c in df_sensea_output.columns if c != "date"]
fig_sensea = line_graph(df_sensea_output, x="date", y=descriptors_sensea)
fig_sensea.show()

In [ ]:
# df_sensea_norm = normalize_acoustic(df_sensea_output)
fig_sensea = line_graph(df_sensea_output, x="date", y=descriptors_sensea, norm=True)
fig_sensea.show()

In [ ]:
df_acoustic = pd.merge(df_otoriver_output, df_sensea_output, on="date", how="outer")
df_acoustic

## Physico-Chimie

Au niveau des données physico-chimiques:
- Physico-chime continue => horaire
- Physico-chimie ponctuelle => hebdo 

In [ ]:
from DBBuilder.datareader import PhCh_Continus_DataReader

datareader_phch_h = PhCh_Continus_DataReader(timefreq="h", renamecolumns=False)
filepath_phch_h = "data/Data_SAM/Bougival/input/phch/phch_continue/phch_continue_20210601-20230831.csv"
datareader_phch_h.metadata

In [ ]:
# df_phch_h = pd.read_excel(filepath_phch_h, decimal=",")
# df_phch_h.to_csv(filepath_phch_h.replace(".xlsx", ".csv"), index=False)
# df_phch_h

In [ ]:
df_phch_h = pd.read_csv(filepath_phch_h.replace(".xlsx", ".csv"))
print("Donnée physico-chimie continue d'entrée")
display(df_phch_h)

In [ ]:
df_phch_h_output = datareader_phch_h.from_dataframe(df=df_phch_h)
print("Donnée physico-chimie continue de sortie")
display(df_phch_h_output)

In [ ]:
cols_todisplay = [c for c in df_phch_h_output.columns if c != "date"]
fig_phch_h = line_graph(df=df_phch_h_output, x="date", y=cols_todisplay, title="Évolution des paramètres physico-chimiques (données normalisées)", norm=True)
fig_phch_h.show() 

## Build DataBase

In [ ]:
from DBBuilder import DBBuilder

lat, long, elev = 48.866,2.133, 23
dbbuilder = DBBuilder(latitude=lat, longitude=long, elevation=elev)

In [ ]:
df_final = pd.merge(df_acoustic, df_phch_h_output, on="date", how="outer")
display(df_final)

In [ ]:
output = dbbuilder.build(input=df_final)

In [ ]:
hourly_df = dbbuilder.rename_columns_with_metadata(df=dbbuilder.hourly_data)

In [ ]:
display(dbbuilder.data)
display(dbbuilder.hourly_data)
display(dbbuilder.daily_data)

In [ ]:
# dbbuilder.data.to_csv("data/Data_SAM/Bougival/output/Bougival_dataset.csv", date_format="%Y-%m-%d %H:%M:%S", index=False)
# dbbuilder.hourly_data.to_csv("data/Data_SAM/Bougival/output/Bougival_hourly_dataset.csv", date_format="%Y-%m-%d %H:%M:%S", index=False)
# dbbuilder.daily_data.to_csv("data/Data_SAM/Bougival/output/Bougival_daily_dataset.csv", date_format="%Y-%m-%d %H:%M:%S", index=False)

In [ ]:
# dbbuilder.export_to_xlsx(filepath="database.xlsx")

In [ ]:
hourly_data = dbbuilder.hourly_data
hourly_data = dbbuilder.rename_columns_with_metadata(df=hourly_data, add_metadata_infos=True)
print(hourly_data.columns.to_list())
sensea_cols = []
otoriver_cols = []
meteo_cols = []
phch_cols = []
for c in hourly_data.columns:
    c = str(c)
    print(c)
    if "sensea" in c:
        print(c)
        sensea_cols.append(c)
    if "otoriver" in c:
        otoriver_cols.append(c)
    if "meteo" in c:
        meteo_cols.append(c)
    if "phch" in c:
        phch_cols.append(c)

In [ ]:
dbbuilder.export_to_csv(folderpath="./data/Data_SAM/", baseFilename="Bougival_dataset")

In [ ]:
daily_data = dbbuilder.daily_data
display(daily_data)
date_interval = [daily_data.date.min(), daily_data.max()]
# "-".join([d.strftime("%Y%m%d") for d in [daily_data.date.min(), daily_data.max()]])
date_interval

In [ ]:
daily_data.date.min().strftime("%Y%m%d")